<a href="https://colab.research.google.com/github/ev1025/GA_LOG_ANALYST/blob/main/Hits%EC%BB%AC%EB%9F%BC_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ipywidgets # tqdm

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


In [ ]:
import pandas as pd
import numpy as np
from pandas import json_normalize
import json
import os
import csv
import ast
from tqdm.notebook import tqdm
from datetime import datetime


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# train_v2가 있는 경로로 이동
os.chdir('C:\\Users\\ENVY\\Desktop\\GA')

**Hits컬럼의 값 뽑기**

In [ ]:
# totals의 hits 값 가져오기위해 데이터 읽어오기
df_index = pd.read_csv('train_v2.csv',
                        usecols=['totals'],  # hits(총 조회수 정보 추출)
                        converters={'totals':ast.literal_eval}) # str형식을 json형식으로 바꿔주기

# json을 columns : 값 형식으로 바꿔주기
df_index_df = json_normalize(df_index['totals']) 
df_index_df['hits'] = df_index_df['hits'].astype(int)

# hits가 100이상이 아닌 컬럼을 뽑아준다.
hit_100_index = np.array(df_index_df[~(df_index_df['hits']>100)].index)
# read_csv의 skiprows는 인덱스에 +1을 해줘야하기 때문에 빠른 계산을 위해 np로 각 index에 +1
hit_100_index +=1

In [ ]:
# hits가 100개 이상인 컬럼은 1541개
len(df_index_df)-len(hit_100_index)

1541

In [ ]:
# totals의 hits(하루 총 조회수)가 100 이상인 고객의 hits컬럼 추출(고유아이디 포함)
df = pd.read_csv('train_v2.csv',
                  header=0, 
                  skiprows=hit_100_index, 
                  usecols=['fullVisitorId','visitStartTime','hits'],
                  dtype = {'fullVisitorId':str},
                  converters={'hits': ast.literal_eval})
df.head()

,fullVisitorId,hits,visitStartTime
0,309482894121265066,"[{'hitNumber': '1', 'time': '0', 'hour': '8', ...",1508166548
1,9291001063810206156,"[{'hitNumber': '1', 'time': '0', 'hour': '14',...",1508190255
2,4459364070259865039,"[{'hitNumber': '1', 'time': '0', 'hour': '13',...",1472847498
3,5911716885988565842,"[{'hitNumber': '1', 'time': '0', 'hour': '19',...",1472870925
4,2940846817699257423,"[{'hitNumber': '1', 'time': '0', 'hour': '14',...",1472851842


In [ ]:
# 값을 넣어줄 빈 데이터프레임 생성, json을 DataFrame화
empty_data = pd.DataFrame()
hit_normal = json_normalize(df['hits'])
hits_max = df_index_df['hits'].max()

# 각 고객별 hits정보(json)를 풀어서 세로로 붙인다.
for i in tqdm(range(hits_max)):
    temp = json_normalize(hit_normal[i])
    empty_data = pd.concat([empty_data,temp],axis=0)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
# 각 값에 고유아이디와 시간을 넣어준다.
hits_100 = pd.merge(df[['fullVisitorId','visitStartTime']],empty_data, how='outer',left_index=True,right_index=True)
hits_100 = hits_100.fillna({'fullVisitorId': df['fullVisitorId'].values, 'visitStartTime':df['visitStartTime'].values})
hits_100['visitStartTime'] =  hits_100['visitStartTime'].apply(lambda x : datetime.utcfromtimestamp(x))
hits_100.head()

,fullVisitorId,visitStartTime,hitNumber,time,hour,minute,isInteraction,isEntrance,product,promotion,experiment,customVariables,customDimensions,customMetrics,type,dataSource,publisher_infos,page.pagePath,page.hostname,page.pageTitle,page.pagePathLevel1,page.pagePathLevel2,page.pagePathLevel3,page.pagePathLevel4,appInfo.screenName,appInfo.landingScreenName,appInfo.exitScreenName,appInfo.screenDepth,exceptionInfo.isFatal,promotionActionInfo.promoIsView,eCommerceAction.action_type,eCommerceAction.step,social.socialNetwork,social.hasSocialSourceReferral,social.socialInteractionNetworkAction,contentGroup.contentGroup1,contentGroup.contentGroup2,contentGroup.contentGroup3,contentGroup.contentGroup4,contentGroup.contentGroup5,contentGroup.previousContentGroup1,contentGroup.previousContentGroup2,contentGroup.previousContentGroup3,contentGroup.previousContentGroup4,contentGroup.previousContentGroup5,referer,transaction.currencyCode,item.currencyCode,contentGroup.contentGroupUniqueViews2,contentGroup.contentGroupUniqueViews3,eventInfo.eventCategory,eventInfo.eventAction,eventInfo.eventLabel,contentGroup.contentGroupUniqueViews1,eCommerceAction.option,promotionActionInfo.promoIsClick,latencyTracking.pageLoadSample,latencyTracking.pageLoadTime,latencyTracking.pageDownloadTime,latencyTracking.redirectionTime,latencyTracking.speedMetricsSample,latencyTracking.serverResponseTime,latencyTracking.domLatencyMetricsSample,latencyTracking.domInteractiveTime,latencyTracking.domContentLoadedTime,latencyTracking.domainLookupTime,latencyTracking.serverConnectionTime,transaction.transactionId,transaction.transactionRevenue,transaction.transactionTax,transaction.transactionShipping,transaction.affiliation,transaction.localTransactionRevenue,transaction.localTransactionTax,transaction.localTransactionShipping,item.transactionId,transaction.transactionCoupon,isExit
0,309482894121265066,2017-10-16 15:09:08,1,0,8,9,True,True,[],"[{'promoId': 'Apparel Row 1', 'promoName': 'Ap...",[],[],[],[],PAGE,web,[],/home,shop.googlemerchandisestore.com,Home,/home,,,,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/google+redesig...,0,True,True,0,1,(not set),No,:,(not set),(not set),(not set),(not set),(not set),(entrance),(entrance),(entrance),(entrance),(entrance),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,309482894121265066,2017-10-16 15:09:08,2,7662,8,9,True,NaN,[],[],[],[],[],[],PAGE,web,[],/signin.html,shop.googlemerchandisestore.com,The Google Merchandise Store - Log In,/signin.html,,,,shop.googlemerchandisestore.com/signin.html,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/google+redesig...,0,True,NaN,0,1,(not set),No,:,(not set),(not set),(not set),(not set),(not set),(not set),(not set),(not set),(not set),(not set),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,309482894121265066,2017-10-16 15:09:08,3,17130,8,9,True,NaN,[],"[{'promoId': 'Apparel Row 1', 'promoName': 'Ap...",[],[],[],[],PAGE,web,[],/store.html,shop.googlemerchandisestore.com,Home,/store.html,,,,shop.googlemerchandisestore.com/store.html,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/google+redesig...,0,True,True,0,1,(not set),No,:,(not set),(not set),(not set),(not set),(not set),(not set),(not set),(not set),(not set),(not set),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,309482894121265066,2017-10-16 15:09:08,4,23462,8,9,True,NaN,[],[],[],[],[],[],PAGE,web,[],/wishlist.html,shop.googlemerchandisestore.com,Your Wishlist,/wishlist.html,,,,shop.googlemerchandisestore.com/wishlist.html,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/google+redesig...,0,True,NaN,0,1,(not set),No,:,(not set),(not set),(not set),(no

In [ ]:
# 첫번째 고유아이디의 hit수 확인
hits_100[hits_100['fullVisitorId']=='309482894121265066']['hitNumber']

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56',
       '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67',
       '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78',
       '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89',
       '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100',
       '101', '102', '103', '104', '105', '106', '107', '108', '109',
       '110', '111', '112', '113', '114', '115', '116', '117', '118',
       '119', '120', '121', '122', '123', '124', '125', '126', '127',
       '128', '129', '130', '131', '132', '133', '134', '135', '136',
       '137', '138', nan], dtype=object)

In [ ]:
# 고유아이디가 알맞게 부여되었음을 확인
hits_100[hits_100['fullVisitorId']=='309482894121265066']['hitNumber'].notna().sum()

138

In [ ]:
hits_100.to_csv('hit_100.csv', index=False)

**chunk로 데이터 뽑기**

In [ ]:
# address에 chunk 저장
# address = pd.read_csv('train_v2.csv', chunksize=10000,
#                        dtype={'fullVisitorId':'str',('date','fulVisitorid', 'visitId', 'visitNumber', 'visitStartTime'):'int'})
# 빈프레임 생성
# train_data = pd.DataFrame()

In [ ]:
# # 1만개씩 분할하여 빈 프레임에 30만개 수집
# i = 0
# for row, chunk in enumerate(address):
#     train_data = pd.concat([train_data, chunk])
#     print(f'{row*10000}개')
#     i+=1
#     if i == 30:
#         break

In [ ]:
# csv 저장할 때(이렇게 안하면 파일깨짐)
# df.to_csv('df.csv', index=False, lineterminator='\n')